## Sun B1 dynamics: nastier than expected...
When we have 3BP dynamics in E-M rotating frame, we naturally forget about $\omega$ as we normalize t* so that $\omega$ = 1 (i.e., $t_1^*$ = 2$\pi$ / (1 siderial period = 27.5days). 
This means omega should change if we want to discuss the dynamics in Sun-B1 frame ($t_2^*$ = 2$\pi$ / 365 days), while we want to stick with $t_1^*$. Therefore, the angular velocity term in the centrifugal and Coriolis force emerges: $\omega_b = 2\pi$ / (365 days / 27.5 days).

Additionally, according to Boudad's PhD thesis (C.19, P.235), the angular velocity of the E-M line should be $\omega_M = |\omega_S|/(1-|\omega_S|) ~= 1.05$, rather than $\omega_S = - t_{siderial}/t_{synodic} ~= -0.93$, which is the angular velocity of the Sun w.r.t. Earth-Moon rotating frame in BCR4P.




### normalization
$t^* = t_{syderial} = 27.5 * 86400$ [s], 

$l^*$ = Earth-Moon distance,

$m^* = m_E + m_M$.

In [ ]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

In [ ]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")
param3b = SailorMoon.dyanmics_parameters()


In [ ]:
"""
Right-hand side expression for state-vector in BCR4BP
# Arguments
    - `du`: cache array of duative of state-vector, mutated
    - `u`: state-vector
    - `p`: parameters, where p = [μ2, μS, as, θ0, ωM, τ, γ, β, mdot, tmax]
        m* : mE + mL (6.0455 * 10^22)
        μ2 : mL / m* (0.012150585609624)
        μS : mS / m* (0.00000303951)
        as : (sun-B1 distance) / l* (388.709677419)
            l* : Earth-moon distance (384,400 km)
        ωM : Earth-Moon line's angular velocity around E-M barycenter
        τ  : thrust magnitude (0~1)
        γ  : thrust angle 1
        β  : thrust angle 2
        mdot : mass-flow rate
        tmax : max thrust
    - `t`: time
"""
function rhs_bcr4bp_sb1frame!(du,u,p,t)
    # unpack state
    x, y, z = u[1], u[2], u[3]
    vx, vy, vz = u[4], u[5], u[6]
    μ2, μS, as, θ0, ωM, ωb = p[1], p[2], p[3], p[4], p[5], p[6]
    τ, γ, β, mdot, tmax = p[7], p[8], p[9], p[10], p[11]
    
    ωM = 0.9251999994040079

    θ = θ0 + ωM * t  # moon angle

    # create Thrust term
#     T = dv_inertial_angles([τ,γ,β])
    T = [0.0, 0.0, 0.0]
    Tx, Ty, Tz = T * tmax / u[7]  # mdot


    # derivatives of positions
    du[1] = u[4]
    du[2] = u[5]
    du[3] = u[6]

    # earth and moon location
    xe = - μ2 *cos(θ) 
    ye = - μ2 *sin(θ)
    ze = 0

    xm = (1-μ2) *cos(θ) 
    ym = (1-μ2) *sin(θ)
    zm = 0
    
#     println(xe, " ", ye, " ", ze)

    # compute distances
    r30 = sqrt((as + x)^2 + y^2 + z^2)      # SC-Sun
    r31 = sqrt((xe - x)^2 + (ye - y)^2 + (ze-z)^2)  # SC-earth
    r32 = sqrt((xm - x)^2 + (ym - y)^2 + (zm-z)^2)  # SC-Moon
    println(r30, " ", r31, " ", r32)

    Fx = -(μS)*(x+as)/r30^3 - (1-μ2)*(x-xe)/r31^3 - μ2*(x-xm)/r32^3 + Tx
    Fy = -(μS)* y / r30^3   - (1-μ2)*(y-ye)/r31^3 - μ2*(y-ym)/r32^3 + Ty
    Fz = -(μS)* z / r30^3   - (1-μ2)*(z-ze)/r31^3 - μ2*(z-zm)/r32^3 + Tz
    println(Fx, " ", Fy, " ", Fz)
    # println(-(μS/(μS+1))*(x-1/(μS+1))/r30^3 , " ", - (1-μ2)/(μS+1)*(x-xe)/r31^3, " ",  - μ2/(μS+1)*(x-xm)/r32^3)
#     println(Fx, " ", Fy, " ", Fz)

    du[4] =  2*ωb*vy + ωb^2*x + Fx
    du[5] = -2*ωb*vx + ωb^2*y + Fy
    du[6] =                   + Fz

    du[7] = -mdot * τ
end

In [ ]:
"""
    transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)

Transform state from Earth-Moon rotating frame to Sun-B1 rotating frame.
Careful with sign of ωs!! (should be negative)
"""
function transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)
    ωm = -ωs
    θm = π - θs
    cos_θm = cos(θm)
    sin_θm = sin(θm)
    C = [
        cos_θm      -sin_θm   0 0       0      0
        sin_θm      cos_θm    0 0       0      0
        0           0         1 0       0      0
        -ωm*sin_θm -ωm*cos_θm 0 cos_θm -sin_θm 0 
         ωm*cos_θm -ωm*sin_θm 0 sin_θm  cos_θm 0
         0          0         0 0       0      1
    ]
    state_conv = C * state    
    return state_conv
end


In [ ]:
function plot_circle(radius, x, y, n=50)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

In [ ]:
θM = 180 / 180 * pi  
ϕ0 = 0.0 # 5.27787565803085   #0.0
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
ωb = 2*pi*(t_synodic - t_sidereal)/t_synodic/t_sidereal*param3b.tstar
print(ωb)
param = [param3b.mu2, param3b.mus, param3b.as, θM, param3b.oml, ωb, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
# initial condition: Halo orbit progapation
lps = SailorMoon.lagrange_points(param3b.mu2)

## set up of initial condition (Lyapunov orbit)
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1) # monodromy eigenvector

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO2(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, 1e-6, Tsit5(), 1e-12, 1e-12
);

# xf in the Earth-moon rotating frame
xf_em = SailorMoon.set_terminal_state2(ϕ0, θf, param3b, LPOArrival)


In [ ]:
# change coordination from EM rot. frame to Sun-B1 frame
θs = π - θM
ωs = param3b.oms
xf_sb1 = vcat(transform_EMrot_to_SunB1(xf_em, θs, ωs), 1.0)
println(xf_sb1)

# propagate ODE
tspan = [0.0, -29]
prob = ODEProblem(rhs_bcr4bp_sb1frame!, xf_sb1, tspan, param)
sol = solve(prob, Tsit5(), reltol=1e-12, abstol=1e-12)

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
plot!(pcart, Array(sol)[1,:], Array(sol)[2,:], color=:blue, linewidth=1.0, label="sol", linestyle=:solid)

moon = plot_circle(1-param3b.mu2, 0.0 , 0.0)
earth = plot_circle(param3b.mu2, 0.0, 0.0)
moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, 0.0, 0.0)


plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

In [ ]:
-1.1226882532375977 + param3b.as